In [16]:
## TODO: play with evaluate
import evaluate
import numpy as np

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer

In [5]:
from src.performance import PerformanceBenchmark, get_performance_benchmark_test
from src.helper_fn import get_dataset, get_dataloader, get_model_for_training

In [33]:
model, tokenizer = get_model_for_training(fine_tuning='no')
train_dataset, test_dataset = get_dataset(tokenizer=tokenizer, cap=None)
train_dataloader, test_dataloader = get_dataloader(train_dataset, test_dataset)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/meninderpurewal/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343/cache-ebc95fccb762f454.arrow
Loading cached processed dataset at /Users/meninderpurewal/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343/cache-7675fc0438ed914c.arrow
Loading cached processed dataset at /Users/meninderpurewal/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343/cache-52ffaee47e2c36b5.arrow
Loading cached processed dataset at /Users/meninderpurewal/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343/cache-cfa1d0b27ba27d7b.arrow
Loading cached processed dataset at /Users/meninderpurewal/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d7

In [6]:
results = {}
results.update(get_performance_benchmark_test(model, tokenizer, 'mps', test_dataloader, 'original_model_pb'))

/Users/meninderpurewal/My Drive/Mikey/Code/BertLora/src/performance.py:32: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  precision = load_metric('precision')
/Users/meninderpurewal/miniconda3/envs/hf2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2023-09-21 08:11:50,433 - src.logger - INFO - Finished original_model_pb run in 6.495651999983238 seconds
2023-09-21 08:11:50,434 - src.logger - INFO - {'original_model_pb': {'precision': 0.2074074074074074, 'f1': 0.23318597111700565}}


In [117]:
model_lora = BertForSequenceClassification.from_pretrained("models/lora", num_labels=3)

OSError: models/lora does not appear to have a file named config.json. Checkout 'https://huggingface.co/models/lora/None' for available files.

In [9]:
results.update(get_performance_benchmark_test(model_top2, tokenizer, 'mps', test_dataloader, 'top2_model_pb'))

2023-09-21 08:15:06,293 - src.logger - INFO - Finished top2_model_pb run in 4.768301375006558 seconds
2023-09-21 08:15:06,293 - src.logger - INFO - {'top2_model_pb': {'precision': 0.7145969498910675, 'f1': 0.7254418487999299}}


In [47]:
model_top = BertForSequenceClassification.from_pretrained("top", num_labels=3)

In [48]:
results.update(get_performance_benchmark_test(model_top, tokenizer, 'mps', test_dataloader, 'top_model_pb'))

2023-09-21 10:24:31,367 - src.logger - INFO - Finished top_model_pb run in 461.30361841700505 seconds
2023-09-21 10:24:31,372 - src.logger - INFO - {'top_model_pb': {'precision': 0.6624318900681477, 'f1': 0.6665939111466307}}


In [53]:
training_args = TrainingArguments(
    output_dir='temp',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    learning_rate=5e-5,
    warmup_ratio=0.1,
    evaluation_strategy="steps",
    save_steps=1_000,
    eval_steps=1_000,
    save_total_limit=5,
    use_mps_device='mps',
    logging_dir='top' + '/logs',
    
    )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [54]:
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

In [55]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=lambda pred: {
            "precision": precision_metric.compute(
                predictions=np.argmax(pred.predictions, axis=-1),
                references=pred.label_ids,
                average="macro",
            )["precision"],
            "f1": f1_metric.compute(
                predictions=np.argmax(pred.predictions, axis=-1),
                references=pred.label_ids,
                average="macro",
            )["f1"],
        },
    )

In [43]:
t = trainer.evaluate()

  0%|          | 0/768 [00:00<?, ?it/s]

/Users/meninderpurewal/miniconda3/envs/hf2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
t

{'eval_loss': 1.1688824892044067,
 'eval_precision': 0.10778248127645718,
 'eval_f1': 0.16289370078740156,
 'eval_runtime': 460.3609,
 'eval_samples_per_second': 26.683,
 'eval_steps_per_second': 1.668}

In [45]:
results.update({'trainer_original_model_full':{'precision': t['eval_precision'], 'f1':t['eval_f1']}})

In [49]:
results 

{'original_model_pb': {'precision': 0.2074074074074074,
  'f1': 0.23318597111700565},
 'top2_model_pb': {'precision': 0.7145969498910675, 'f1': 0.7254418487999299},
 'trainer_original_model': {'precision': 0.2074074074074074,
  'f1': 0.23318597111700565},
 'trainer_top2_model': {'precision': 0.7145969498910675,
  'f1': 0.7254418487999299},
 'trainer_top2_model_full': {'precision': 0.6666778796531642,
  'f1': 0.6699837856293124},
 'trainer_original_model_full': {'precision': 0.10778248127645718,
  'f1': 0.16289370078740156},
 'top_model_pb': {'precision': 0.6624318900681477, 'f1': 0.6665939111466307}}

In [106]:
from src.helper_fn import get_model_for_training
model, tokenizer = get_model_for_training(fine_tuning='no')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

In [94]:
from peft import LoraConfig, TaskType, get_peft_model

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/meninderpurewal/miniconda3/envs/hf2/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [110]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=1,
    lora_alpha=1,
    lora_dropout=0.1,
)

In [111]:
model = get_peft_model(model, lora_config)

In [112]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(28996, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(
                    in_features=768, out_features=768, bias=True
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_

In [113]:
trainable_params = filter(lambda p: p.requires_grad, model.parameters())

In [114]:
num_trainable_params = sum(p.numel() for p in trainable_params)

In [115]:
num_trainable_params

41478